In [2]:
import pandas as pd
# import data and display df

path_all_perth = "C:/Users/jess9/Desktop/Personal/coding/RealEstate_Scraping/all_perth.csv"
df = pd.read_csv(path_all_perth)

df


,Address,Link,Bedrooms,Bathrooms,Parking,Area,Type,Suburb,Scraped Date,Days on Market,Asking Price
0,"16/181 Oxford Street, Leederville",https://www.realestate.com.auhttps://www.reale...,1,1,1,0,1,perth,2025-10-01,NaN,400000
1,"30/124 Terrace Road, Perth",https://www.realestate.com.auhttps://www.reale...,1,1,0,33,1,perth,2025-10-01,NaN,315000
2,"13/34 East Parade, East Perth",https://www.realestate.com.auhttps://www.reale...,1,1,1,0,1,perth,2025-10-01,NaN,500
3,"704/237 Adelaide Terrace, Perth",https://www.realestate.com.auhttps://www.reale...,1,1,1,63,1,perth,2025-10-01,NaN,499000
4,"Address available on request, East Perth",https://www.realestate.com.auhttps://www.reale...,1,1,1,40,1,perth,2025-10-01,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
372,"110/326-328 Burwood Hwy, Burwood",https://www.realestate.com.auhttps://www.reale...,1,1,1,0,1,mount,2025-10-01,NaN,370000407000
373,"G07/40 Bettina Street, Clayton",https://www.realestate.com.auhttps://www.reale...,2,1,1,0,2,mount,2025-10-01,NaN,480000520000
374,"208/6 Dalgety, Oakleigh",https://www.realestate.com.auhttps://www.reale...,1,1,1,64,1,mount,2025-10-01,NaN,430000450000
375,"111/379-381 Burwood Highway, Burwood",https://www.realestate.com.auhttps://www.reale...,2,2,1,61,2,mount,2025-10-01,NaN,465000480000


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Address         377 non-null    object 
 1   Link            377 non-null    object 
 2   Bedrooms        377 non-null    int64  
 3   Bathrooms       377 non-null    int64  
 4   Parking         377 non-null    int64  
 5   Area            377 non-null    int64  
 6   Type            377 non-null    int64  
 7   Suburb          377 non-null    object 
 8   Scraped Date    377 non-null    object 
 9   Days on Market  0 non-null      float64
 10  Asking Price    377 non-null    int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 32.5+ KB


In [22]:
df = df.convert_dtypes()


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Address         377 non-null    string
 1   Link            377 non-null    string
 2   Bedrooms        377 non-null    Int64 
 3   Bathrooms       377 non-null    Int64 
 4   Parking         377 non-null    Int64 
 5   Area            377 non-null    Int64 
 6   Type            377 non-null    Int64 
 7   Suburb          377 non-null    string
 8   Scraped Date    377 non-null    string
 9   Days on Market  0 non-null      Int64 
 10  Asking Price    377 non-null    Int64 
dtypes: Int64(7), string(4)
memory usage: 35.1 KB


In [26]:
df

,Address,Link,Bedrooms,Bathrooms,Parking,Area,Type,Suburb,Scraped Date,Days on Market,Asking Price
0,"16/181 Oxford Street, Leederville",https://www.realestate.com.auhttps://www.reale...,1,1,1,0,1,perth,2025-10-01,<NA>,400000
1,"30/124 Terrace Road, Perth",https://www.realestate.com.auhttps://www.reale...,1,1,0,33,1,perth,2025-10-01,<NA>,315000
2,"13/34 East Parade, East Perth",https://www.realestate.com.auhttps://www.reale...,1,1,1,0,1,perth,2025-10-01,<NA>,500
3,"704/237 Adelaide Terrace, Perth",https://www.realestate.com.auhttps://www.reale...,1,1,1,63,1,perth,2025-10-01,<NA>,499000
4,"Address available on request, East Perth",https://www.realestate.com.auhttps://www.reale...,1,1,1,40,1,perth,2025-10-01,<NA>,0
...,...,...,...,...,...,...,...,...,...,...,...
372,"110/326-328 Burwood Hwy, Burwood",https://www.realestate.com.auhttps://www.reale...,1,1,1,0,1,mount,2025-10-01,<NA>,370000407000
373,"G07/40 Bettina Street, Clayton",https://www.realestate.com.auhttps://www.reale...,2,1,1,0,2,mount,2025-10-01,<NA>,480000520000
374,"208/6 Dalgety, Oakleigh",https://www.realestate.com.auhttps://www.reale...,1,1,1,64,1,mount,2025-10-01,<NA>,430000450000
375,"111/379-381 Burwood Highway, Burwood",https://www.realestate.com.auhttps://www.reale...,2,2,1,61,2,mount,2025-10-01,<NA>,465000480000


In [ ]:
df.sort_values(by=[df.columns[2], df.columns[10]], ascending=[False,False], inplace=True)

df

,Address,Link,Bedrooms,Bathrooms,Parking,Area,Type,Suburb,Scraped Date,Days on Market,Asking Price
224,"Address available on request, Mandurah",https://www.realestate.com.auhttps://www.reale...,5,2,8,760,5,mandurah,2025-10-01,<NA>,0
223,"197 Inlet Boulevard, South Yunderup",https://www.realestate.com.auhttps://www.reale...,4,2,2,330,4,mandurah,2025-10-01,<NA>,550000
200,"Address available on request, Mandurah",https://www.realestate.com.auhttps://www.reale...,4,2,0,0,4,mandurah,2025-10-01,<NA>,549990
157,"Address available on request, Hilbert",https://www.realestate.com.auhttps://www.reale...,4,2,2,0,4,armadale,2025-10-01,<NA>,549927
221,"Address available on request, Coodanup",https://www.realestate.com.auhttps://www.reale...,4,2,2,315,4,mandurah,2025-10-01,<NA>,549888
...,...,...,...,...,...,...,...,...,...,...,...
60,"205/45 MALCOLM STREET, West Perth",https://www.realestate.com.auhttps://www.reale...,0,1,0,0,1,perth,2025-10-01,<NA>,329000
96,"205/45 MALCOLM STREET, West Perth",https://www.realestate.com.auhttps://www.reale...,0,1,0,0,1,subiaco,2025-10-01,<NA>,329000
286,"205/45 MALCOLM STREET, West Perth",https://www.realestate.com.auhttps://www.reale...,0,1,0,0,1,perth,2025-10-01,<NA>,329000
30,"506/8 Adelaide Terrace, East Perth",https://www.realestate.com.auhttps://www.reale...,0,1,1,0,1,perth,2025-10-01,<NA>,0


In [37]:
df1 = df['Days on Market'].drop

In [38]:
df1

<bound method Series.drop of 224    <NA>
223    <NA>
200    <NA>
157    <NA>
221    <NA>
       ... 
60     <NA>
96     <NA>
286    <NA>
30     <NA>
256    <NA>
Name: Days on Market, Length: 377, dtype: Int64>